In [1]:
%cd ..
from src.human_taxonomy_migrations import *
from src.hierarchy import *
from src.common import *
from sklearn.model_selection import train_test_split
%cd -

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


**To DO**

1. Combine all labeling data into one dataframe
2. Add one-hot-encoding for human-group
3. Split into training and testing dataframes. Upload to s3
4. Copy all images that exist in dataframes over to images folder (in the folder with dfs)
5. Copy the version of the taxonomy that is used for training to s3 
6. Do we want to incorporate SMOTE here when creating the dataframes?

s3://data-science-training-datasets/product-image/mturk-30k-plus-10k/ 
look in this folder for example

see create-dataset-from-manifests ipynb but this will need to be updated. some of this notebook will replace that 


In [2]:
old_category_30k = create_dataset_from_manifest('s3://data-science-training-output/product-image/mturk-30k-full/output.manifest',
                             "../files/label-taxonomy-final.json", 
                             '../files/original-to-final-taxonomy-map.json'
                             , '../files/product_image_label_to_training_category_mapping.csv'
                             , 'mturk-30k-full'
                             , '..//files/human-group-taxonomy.json')

In [4]:
len(old_category_30k['one_hot_list'][0])

57

In [5]:
old_category_10k = create_dataset_from_manifest('s3://data-science-training-output/product-image/mturk-10k-full/output.manifest',
                             "../files/label-taxonomy-final.json", 
                             '../files/original-to-final-taxonomy-map.json',
                            '../files/product_image_label_to_training_category_mapping.csv',
                            'mturk-10k-full',
                            '..//files/human-group-taxonomy.json')

In [6]:
len(old_category_10k.one_hot_list[0])

57

In [7]:
new_category_20k = create_dataset_from_manifest("s3://data-science-labeling-output/product-insight/new-category-20k-mturk/manifests/output/output.manifest",
                             "../files/label-taxonomy-final.json", 
                             None, 
                            '../files/product_image_label_to_training_category_mapping.csv',
                            'new-category-20k-mturk',
                             '..//files/human-group-taxonomy.json')

In [9]:
len(new_category_20k.one_hot_list[0])

57

In [10]:
new_manifest = old_category_30k.append(old_category_10k, ignore_index=True).append(new_category_20k, ignore_index=True)

In [11]:
new_manifest.head(2)

,product_id,target_image_url,title,description,confidence,original_label,human_group,category_id,category_name,one_hot_list,human_group_id,human_group_one_hot_list,image
0,147568898,s3://data-science-product-image/datasets/produ...,Gradina Block Heel Bootie,Free shipping and returns on Vince Camuto Grad...,0.666667,31,Women,15,Boots,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",147568898.jpg
1,142331219,s3://data-science-product-image/datasets/produ...,Seamed & Structured Denim Jacket,Free People - Seamed & Structured Denim Jacket...,1.000000,9,Women,3,Coats & Jackets,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",142331219.jpg


In [12]:
print('Length of 30k Old Manifest:',len(old_category_30k))
print('Length of 10k Old Manifest:',len(old_category_10k))
print('Length of 20k Mew Manifest:',len(new_category_20k))
print('Length of New Combined Manifest:',len(new_manifest))

Length of 30k Old Manifest: 28925
Length of 10k Old Manifest: 10978
Length of 20k Mew Manifest: 16651
Length of New Combined Manifest: 56554


In [13]:
new_manifest.category_name.value_counts()

Tops                     5573
Dresses                  3647
Coats & Jackets          3468
Kitchen                  3148
Boots                    2875
Makeup                   2759
Hoodies & Sweatshirts    2237
Electronics              1949
Skincare                 1901
Sneakers & Athletic      1880
Jeans                    1597
Indoor Furniture         1466
Bags                     1464
Leggings                 1432
Toys & Games             1248
Cardigans                1218
Shorts                   1193
Hats                     1188
Bedding & Linens         1080
Skirts                    872
Baby Gear                 863
Sleepwear                 846
Sandals                   834
Intimate Wear             807
Eyewear                   785
Food                      782
Jumpers & Rompers         755
Hair                      753
Swimwear                  651
Necklaces                 647
Earrings                  646
Bracelets                 593
Closed-Toe Flats          512
Watches   

In [14]:
#need list of image paths - need to know where to copy images from
images_uris = list(new_manifest['target_image_url'].apply(lambda x: "/".join(x.split("/")[:-1])).value_counts().index)

#are these correct? if not add the correct ones
images_uris.append('s3://data-science-training-datasets/product-image/mturk-30k-plus-10k/images')
images_uris.remove('s3://data-science-product-image/datasets/product-image-mturk-30k/images')
images_uris.remove('s3://data-science-product-image/datasets/product-image-mturk-10k/images')
images_uris

['s3://data-science-labeling-input/product-insight/new-category-20k/images',
 's3://data-science-training-datasets/product-image/mturk-30k-plus-10k/images']

In [15]:
#taken from create-dataset-from-manifests ipynb
def copy_images_to_new_s3_folder(combined_manifest, s3_base_path, copy_images=False):
    if copy_images:
        s3_images_path = s3_base_path + "/images"
        for images_uri in images_uris:
            !aws s3 cp {images_uri} {s3_images_path} --recursive > /dev/null

In [16]:
#update this to True when you need to copy images
copy_images_to_new_s3_folder(new_manifest, "s3://data-science-training-datasets/product-image/mturk-50k", False)
copy_images_to_new_s3_folder(new_manifest, "s3://data-science-training-datasets/product-image/mturk-50k-v2", False)

In [17]:
final_manifest = final_dataset_updates(new_manifest, 'data-science-training-datasets', "product-image/mturk-50k/images", 100)

In [18]:
len(final_manifest)

51700

In [19]:
final_manifest.tail(1)

,product_id,target_image_url,title,description,confidence,original_label,human_group,category_id,category_name,one_hot_list,human_group_id,human_group_one_hot_list,image
52184,ffd9c6aced2f5819fa63151fed0e1007,s3://data-science-training-datasets/product-im...,Patchology Lip Renewal FlashPatch 5-Minute Hyd...,Patchology Lip Renewal FlashPatch 5-Minute Hyd...,0.666667,81,None,51,Makeup,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1.0, 0.0, 0.0, 0.0, 0.0]",ffd9c6aced2f5819fa63151fed0e1007.jpg


In [20]:
#taken from create-dataset-from-manifests ipynb
def write_dataset_to_s3(combined_manifest, s3_base_path, test_split=0.1):
    training, testing = train_test_split(final_manifest, test_size=0.1)
    training_path = "training_set.json"
    testing_path  = "testing_set.json"
    training.to_json(training_path)
    testing.to_json(testing_path)
    s3_dataframe_path = s3_base_path + "/dataframes/"
    training_s3_path = s3_dataframe_path + "training.json"
    testing_s3_path  = s3_dataframe_path + "testing.json"
    !aws s3 cp {training_path} {training_s3_path}
    !rm {training_path}
    !aws s3 cp {testing_path} {testing_s3_path}
    !rm {testing_path}

In [21]:
# write_dataset_to_s3(final_manifest, "s3://data-science-training-datasets/product-image/mturk-50k")

In [22]:
write_dataset_to_s3(final_manifest, "s3://data-science-training-datasets/product-image/mturk-50k-v2")

upload: ./training_set.json to s3://data-science-training-datasets/product-image/mturk-50k-v2/dataframes/training.json
upload: ./testing_set.json to s3://data-science-training-datasets/product-image/mturk-50k-v2/dataframes/testing.json


In [23]:
final_manifest_dummy = final_dataset_updates(new_manifest, 'data-science-training-datasets', "product-image/mturk-50k-dummy/images", 100)
write_dataset_to_s3(final_manifest_dummy.head(100), "s3://data-science-training-datasets/product-image/mturk-50k-dummy")

upload: ./training_set.json to s3://data-science-training-datasets/product-image/mturk-50k-dummy/dataframes/training.json
upload: ./testing_set.json to s3://data-science-training-datasets/product-image/mturk-50k-dummy/dataframes/testing.json


**Upload Taxonomy**
- Uploaded the transformed taxonomy

In [24]:
#new taxonomy using aggregated categories and transformed so leafs first
taxonomy=load_taxonomy('..//files/label-taxonomy-final.json')
taxonomy[0]

H=Hierarchy(taxonomy, children_key='subcategories')
label_to_training_mapping = pd.read_csv('../files/product_image_label_to_training_category_mapping.csv')
label_to_final_cat = {}
label_to_final_cat_name = {}

for dic in H.taxonomy:
    label_category_id = dic['id']
    training_category_id = int(label_to_training_mapping[label_to_training_mapping['label_category_id']==label_category_id]['training_category_id'].values[0])
    training_category_name = str(label_to_training_mapping[label_to_training_mapping['label_category_id']==label_category_id]['training_category_name'].values[0])
    dic['id'] = training_category_id
    dic['name'] = training_category_name
    label_to_final_cat[label_category_id] = training_category_id
    label_to_final_cat_name[label_category_id] = training_category_name

    if 'parent' in dic.keys():
        label_parent_category_id = dic['parent']
        training_parent_category_id = int(label_to_training_mapping[label_to_training_mapping['label_category_id']==label_parent_category_id]['training_category_id'].values[0])
        dic['parent'] = training_parent_category_id

    new_subcategories_list = []
    for subcat in dic['subcategories']:
        new_subcat = int(label_to_training_mapping[label_to_training_mapping['label_category_id']==subcat]['training_category_id'].values[0])
        if new_subcat!=training_category_id:
            new_subcategories_list.append(new_subcat)
    dic['subcategories'] = list(set(new_subcategories_list))  

    if dic['subcategories']==[]:
        dic['is_leaf']=True
    else:
        dic['is_leaf']=False
H.taxonomy = [i for n, i in enumerate(H.taxonomy) if i not in H.taxonomy[n + 1:]]
H.taxonomy = [i for i in H.taxonomy if (i['name']=='Main') or (i['parent']!=i['id'])]  
H.taxonomy = sorted(H.taxonomy, key = lambda i: i['id'])

key_order = ['id','name','subcategories','parent','is_leaf']
new_taxonomy = []

for tax_dict in H.taxonomy:
    for k in key_order:
        tax_dict = {k : tax_dict[k] for k in key_order if k in tax_dict}
    new_taxonomy.append(tax_dict)

with open('taxonomy.json', 'w') as f:
    json.dump(new_taxonomy, f)
    
!aws s3 cp 'taxonomy.json' "s3://data-science-training-datasets/product-image/mturk-50k-dummy/"
!aws s3 cp 'taxonomy.json' "s3://data-science-training-datasets/product-image/mturk-50k-v2/"


upload: ./taxonomy.json to s3://data-science-training-datasets/product-image/mturk-50k-dummy/taxonomy.json
upload: ./taxonomy.json to s3://data-science-training-datasets/product-image/mturk-50k-v2/taxonomy.json


In [36]:
#previous taxonomy for 50k training (non-v2 version)
# taxonomy=load_taxonomy('..//files/label-taxonomy-final.json')
# taxonomy[0]

# H=Hierarchy(taxonomy, children_key='subcategories')
# key_order = ['id','name','subcategories','parent','is_leaf']
# new_taxonomy = []

# for tax_dict in H.taxonomy:
#     for k in key_order:
#         tax_dict = {k : tax_dict[k] for k in key_order if k in tax_dict}
# #     if (tax_dict['id'] in category_ids) or tax_dict['is_leaf']==False:
#     new_taxonomy.append(tax_dict)

# with open('taxonomy.json', 'w') as f:
#     json.dump(new_taxonomy, f)

# !aws s3 cp 'taxonomy.json' "s3://data-science-training-datasets/product-image/mturk-50k/"

# # !rm 'taxonomy.json'

**EDA: What does combined data look like?**

In [19]:
human_groups = pd.DataFrame(new_manifest.groupby('human_group').count()['product_id'])
human_groups['%'] = human_groups/human_groups.sum()
human_groups

,product_id,%
human_group,,
Baby,1102,0.021045
Kids,2464,0.047054
Men,2424,0.046290
None,16696,0.318839
Women,29679,0.566772


In [25]:
categories = pd.DataFrame(new_manifest.groupby('category_name').count()['product_id'])
categories['%'] = categories/categories.sum()
categories.sort_values(by='%', ascending=False).head(30)

,product_id,%
category_name,,
Coats & Jackets,3468,0.066227
Kitchen,3148,0.060116
Casual Dresses,3110,0.059391
Makeup,2759,0.052688
Hoodies & Sweatshirts,2237,0.042719
Electronics,1949,0.037220
Skincare,1901,0.036303
Sneakers & Athletic,1880,0.035902
Jeans,1597,0.030497


In [27]:
categories.sort_values(by='%', ascending=False).to_csv('categories.csv')

**Comparing Distributions Between Training and Testing Dataframes**

In [91]:
!aws s3 cp s3://data-science-training-datasets/product-image/mturk-50k/dataframes/testing.json testing.json

download: s3://data-science-training-datasets/product-image/mturk-50k/dataframes/testing.json to ./testing.json


In [92]:
!aws s3 cp s3://data-science-training-datasets/product-image/mturk-50k/dataframes/training.json training.json

download: s3://data-science-training-datasets/product-image/mturk-50k/dataframes/training.json to ./training.json


In [94]:
testing_df = pd.read_json('testing.json')
training_df = pd.read_json('training.json')

training_df_counts = pd.DataFrame(training_df.groupby(['category_id','category_name']).count()['product_id'])
training_df_counts['total'] = training_df_counts['product_id'].sum(axis=0)
training_df_counts['pct_total'] = training_df_counts['product_id']/training_df_counts['total']
training_df_counts = training_df_counts.reset_index()

testing_df_counts = pd.DataFrame(testing_df.groupby(['category_id','category_name']).count()['product_id'])
testing_df_counts['total'] = testing_df_counts['product_id'].sum(axis=0)
testing_df_counts['pct_total'] = testing_df_counts['product_id']/testing_df_counts['total']
testing_df_counts = testing_df_counts.reset_index()

combined_dfs = pd.merge(training_df_counts, testing_df_counts, how='left', left_on='category_id', right_on='category_id')
combined_dfs.to_csv('combined_dfs.csv')

,category_id,category_name,product_id,total,pct_total
0,0,Jeans,1379,44542,0.03096
